In [1]:
import pickle
import random
import numpy as np
import pandas as pd
import json
from itertools import groupby, chain, permutations, combinations, combinations_with_replacement
from collections import Counter

In [2]:
reg_model = open('final_reg_model.sav', 'rb')
reg = pickle.load(reg_model)

In [3]:
print(reg.feature_importances_)

[2.41929347e-01 1.43612345e-01 7.83580597e-04 1.01204220e-04
 5.51466436e-04 1.13638592e-04 3.71696041e-04 1.03751745e-04
 3.71916972e-04 6.77853381e-06 6.03588345e-04 7.50785656e-05
 2.19364171e-04 9.87194975e-06 1.86194526e-04 2.39813158e-07
 2.12815938e-05 1.41306741e-07 2.55752461e-04 0.00000000e+00
 1.44637358e-03 8.91452310e-05 1.18383149e-03 3.00839551e-04
 1.24860120e-03 4.77289095e-04 1.04717745e-03 1.09367934e-04
 1.17926897e-03 2.78946344e-04 8.70615991e-04 1.80979634e-04
 7.37889822e-04 2.31624847e-05 8.28889389e-05 8.55959545e-06
 1.23882953e-03 0.00000000e+00 2.86183956e-03 7.29824514e-05
 2.61600130e-03 5.46324888e-04 3.10927582e-03 5.93263334e-04
 3.16268441e-03 1.02182841e-03 2.96723129e-03 6.33471286e-04
 2.99187256e-03 7.01051380e-04 3.38401341e-03 8.47366506e-04
 9.19210283e-04 3.57706625e-04 3.08419419e-03 7.10812975e-04
 1.05666925e-03 3.05657170e-04 7.06847372e-04 1.78276506e-04
 2.67623485e-04 7.11753619e-04 8.07633118e-04 3.68518700e-04
 6.17972949e-04 1.184299

In [4]:
# Generate all possible item combinations
from itertools import combinations_with_replacement 
def generate_item_combinations(items_list):
    combinations = combinations_with_replacement(items_list,3)
    res = []
    for combination in combinations:
        combination = tuple(x for x in combination)
        # check if tuple is in ascending order
        if(combination[0] <= combination[1] and combination[1] <= combination[2]):
            # check if there are no more than 1 basic items in the tuple 
            # note: 0 is considered as a combined item since 1, 2 or even all 3 item slots can be empty
            if(np.sum(np.floor([(x/10) if x else 1 for x in combination]) == 0) <= 1):
                res.append(combination)
    return res

In [5]:
items_list = [10 * i + j for i in range(10) for j in range(i,10)]
item_combinations_list = generate_item_combinations(items_list)
random.sample(item_combinations_list, 10)

[(7, 16, 57),
 (0, 15, 66),
 (15, 23, 77),
 (3, 37, 45),
 (14, 24, 29),
 (4, 12, 29),
 (12, 37, 89),
 (14, 48, 67),
 (34, 57, 89),
 (8, 23, 24)]

In [6]:
def vectorise_items(items):
    item_index = {x:y for x,y in zip(range(1,10),range(0,18,2))}
    res = {}
    for i,item in enumerate(items):
        item_str = str(item)
        vect = np.zeros([18])
        for i in item_str:
            if i != '0':
                first_index = item_index[int(i)]
                if(not vect[first_index]):
                    vect[first_index] = 1
                else:
                    vect[first_index+1] = 1
        res[item] = vect
    res[0] = np.zeros([18])
    return res
# Vector embeddings for all item combinations
# Convert a list of items to vectors
item_vector_dict = vectorise_items(items_list)
def item_vector_lookup(item_list, d=item_vector_dict):
    res = []
    for items in item_list:
        temp_res = []
        for item in items:
            temp_res = temp_res + d[item].tolist()
        res.append(temp_res)
    return res
item_combinations_vector = item_vector_lookup(item_combinations_list)

In [7]:
item_combinations_vector = np.asarray(item_combinations_vector)

In [8]:
# Get n item recommendations and their predicted rank by running random forest model on all possible item combinations
def get_n_item_recommendations(model, item_combinations_list, item_combinations_vector, input_vector):
    # Duplicate input vector N times, N = #total number of item combinations
    input_vector_stretched = np.tile(input_vector,(item_combinations_vector.shape[0],1))
    # Add every item combination vector to the stretched input vector
    input_vector_with_item_combo = np.insert(input_vector_stretched,3,item_combinations_vector.transpose(),axis=1)
    # Predict rank
    results = reg.predict(input_vector_with_item_combo)
    # sort indices by predicted rank (descending)
    sorted_items_index = np.argsort(results)
    # Return n item combinations with the highest predicted ranks
    return np.asarray([(item_combinations_list[x], results[x]) for x in sorted_items_index], dtype = object)

# Get the highest average predicted rank of each item by using the model on all item combos
def get_highest_average_rank_items(model, item_combinations_list, item_combinations_vector, input_vector):
    # Duplicate input vector N times, N = #total number of item combinations
    input_vector_stretched = np.tile(input_vector,(item_combinations_vector.shape[0],1))
    # Add every item combination vector to the stretched input vector
    input_vector_with_item_combo = np.insert(input_vector_stretched,3,item_combinations_vector.transpose(),axis=1)
    # Predict rank
    results = reg.predict(input_vector_with_item_combo)
    sum_rank_dict = {}
    for i, rank in enumerate(results):
        for item in item_combinations_list[i]:
            if item not in sum_rank_dict:
                sum_rank_dict[item] = [rank, 1]
            else:
                sum_rank_dict[item][0] += rank
                sum_rank_dict[item][1] += 1
    avg_rank_dict = {}
    for k, v in sum_rank_dict.items():
        avg_rank_dict[k] = 1.0 * v[0] / v[1]
    return avg_rank_dict

In [9]:
with open('full_mapping.json') as json_data:
    fullmap = json.load(json_data)
item_mapping = fullmap['item_mapping_json']
item_mapping['0'] = 'None'

In [10]:
# Get item names for a tuple of item numbers
def get_item_names(item_combo, item_mapping = item_mapping):
    return tuple(item_mapping.get(str(item)) for item in item_combo)

In [11]:
combined_df = pd.read_pickle("final_data_frame_one_hot")
combined_df = combined_df.drop(columns = ["stage"])

In [12]:
col_name = list(combined_df.drop(columns = ["rank"]).columns)
col_importance = list(reg.feature_importances_)
feature_importance = [name for (value, name) in sorted(zip(col_importance, col_name), key = lambda x: x[0], reverse = True)]
print(feature_importance)

['gold_spent', 'level', 'Chrono', 'DarkStar', 'Vanguard', 'StarGuardian', 'Celestial', 'stage_5', 'Protector', 'Cybernetic', 'Battlecast', 'Mystic', 'Blademaster', 'Brawler', 'SpacePirate', 'Astro', 'Sniper', 'Sorcerer', 'Rebel', 'Infiltrator', 'Blaster', 'ManaReaver', 'stage_7', 'MechPilot', 'Demolitionist', 'stage_4', 'stage_6', 'Starship', 'Paragon', 'item_index49', 'item_index43', 'item_index41', 'item_index53', 'item_index47', 'item_index45', 'item_index37', 'stage_3', 'item_index39', 'Mercenary', 'TFT3_Rakan', 'TFT3_JarvanIV', 'item_index19', 'item_index23', 'item_index35', 'item_index21', 'TFT3_Zoe', 'item_index27', 'stage_2', 'TFT3_Poppy', 'TFT3_XinZhao', 'TFT3_Ahri', 'item_index25', 'item_index44', 'TFT3_Xayah', 'TFT3_Graves', 'TFT3_Fiora', 'TFT3_Neeko', 'item_index51', 'TFT3_Leona', 'TFT3_Shen', 'item_index29', 'item_index50', 'TFT3_Illaoi', 'TFT3_Caitlyn', 'item_index1', 'item_index31', 'TFT3_Blitzcrank', 'item_index54', 'TFT3_Ashe', 'item_index48', 'TFT3_Malphite', 'TFT3_Ja

In [91]:
# Get a random row of combined_df and remove all item indices
not_item_index = list(chain(range(3),range(58,combined_df.shape[1])))
#x_single = combined_df.iloc[663,not_item_index]
row = random.randint(0,combined_df.shape[0])
x_single = combined_df.iloc[row,~combined_df.columns.str.match(r'^(item|rank)')]
print(x_single)

def get_row_info(row, x_single):
    character = 'None'
    tft = list(combined_df.iloc[row, combined_df.columns.str.match(r'^TFT3_')].keys())
    stages = list(combined_df.iloc[row, combined_df.columns.str.match(r'^stage')].keys())
    gold = list(combined_df.iloc[row, combined_df.columns.str.match(r'^gold_spent$')].keys())
    levels = list(combined_df.iloc[row, combined_df.columns.str.match(r'^level$')].keys())
    traits = list(set(x_single.keys()) - set(tft) - set(stages) - set(gold) - set(levels))
    for key in tft:
        if x_single[key] == 1:
            character = key
            break
    stage = 'None'
    for key in stages:
        if x_single[key] == 1:
            stage = key
            break
    gold_spent = x_single[gold][0]
    level = x_single[levels][0]
    trait_dict = {}
    for key in traits:
        if x_single[key] > 0:
            trait_dict[key] = x_single[key]
    info_dict = {'Character': character[5:],
                 'Stage': stage,
                 'Gold_spent': gold_spent,
                 'Level': level,
                 'Traits': trait_dict}
    return info_dict

row_info = get_row_info(row, x_single)
input_vector = x_single.to_numpy()

gold_spent         36
level         5.33333
TFT3_Ahri           0
TFT3_Annie          0
TFT3_Ashe           0
               ...   
stage_4             0
stage_5             0
stage_6             0
stage_7             0
stage_8             0
Name: 1854662, Length: 91, dtype: object


In [92]:
recommended_items = get_n_item_recommendations(reg, item_combinations_list, item_combinations_vector, input_vector)
for k, v in row_info.items():
    print(f'{k}: {v}')
recommended_items

Character: Riven
Stage: stage_3
Gold_spent: 36.0
Level: 5.333333333333333
Traits: {'Brawler': 2.0, 'Chrono': 4.0, 'Blaster': 2.0, 'Cybernetic': 3.0, 'Blademaster': 2.0, 'Sniper': 1.0}


array([[(22, 33, 88), 3.73],
       [(11, 33, 88), 3.73],
       [(11, 66, 88), 3.74],
       ...,
       [(68, 99, 99), 4.19],
       [(22, 99, 99), 4.2],
       [(99, 99, 99), 4.24]], dtype=object)

In [93]:
def get_best_item(recommended_items):
    max_rank = recommended_items[0][1]
    max_rank_items = []
    for items,rank in recommended_items:
        if(rank == max_rank):
            max_rank_items += list(items)
    c = Counter(max_rank_items)
    return c.most_common(1)[0][0]

for k, v in row_info.items():
    print(f'{k}: {v}')
print('\nRecommendation: ' + item_mapping.get(str(get_best_item(recommended_items))))

Character: Riven
Stage: stage_3
Gold_spent: 36.0
Level: 5.333333333333333
Traits: {'Brawler': 2.0, 'Chrono': 4.0, 'Blaster': 2.0, 'Cybernetic': 3.0, 'Blademaster': 2.0, 'Sniper': 1.0}

Recommendation: RabadonsDeathcap


In [94]:
for k, v in row_info.items():
    print(f'{k}: {v}')
recommended = [get_item_names(x) for x in recommended_items[:,0]]
print('\nRecommendations:')
for rec in recommended[:10]:
    print(rec)

Character: Riven
Stage: stage_3
Gold_spent: 36.0
Level: 5.333333333333333
Traits: {'Brawler': 2.0, 'Chrono': 4.0, 'Blaster': 2.0, 'Cybernetic': 3.0, 'Blademaster': 2.0, 'Sniper': 1.0}

Recommendations:
('RapidFirecannon', 'RabadonsDeathcap', 'ForceofNature')
('Deathblade', 'RabadonsDeathcap', 'ForceofNature')
('Deathblade', 'DragonsClaw', 'ForceofNature')
('Deathblade', 'BrambleVest', 'ForceofNature')
('RapidFirecannon', 'BrambleVest', 'ForceofNature')
('RapidFirecannon', 'DragonsClaw', 'ForceofNature')
('Deathblade', 'LastWhisper', 'RabadonsDeathcap')
('Deathblade', 'GiantSlayer', 'RabadonsDeathcap')
('Deathblade', 'HextechGunblade', 'RabadonsDeathcap')
('RapidFirecannon', 'ForceofNature', 'ForceofNature')


In [95]:
for k, v in row_info.items():
    print(f'{k}: {v}')
highest_items = get_highest_average_rank_items(reg, item_combinations_list, item_combinations_vector, input_vector)
situational_trait_items = {'BladeoftheRuinedKing', 'InfiltratorsTalons', 'RebelMedal', 'CelestialOrb', 'BattlecastArmor', 'StarGuardiansCharm', 'ProtectorsChestguard', 'DarkStarsHeart'}
print('\nRecommendations:')
for tup in sorted([(item_mapping.get(str(item)), avg_rank) for item, avg_rank in highest_items.items()], key = lambda x: x[1]):
    #if tup[0] not in situational_trait_items:
    print(tup)

Character: Riven
Stage: stage_3
Gold_spent: 36.0
Level: 5.333333333333333
Traits: {'Brawler': 2.0, 'Chrono': 4.0, 'Blaster': 2.0, 'Cybernetic': 3.0, 'Blademaster': 2.0, 'Sniper': 1.0}

Recommendations:
('ForceofNature', 3.809509993552621)
('ProtectorsChestguard', 3.8663765312702094)
('CelestialOrb', 3.8718891038040457)
('RabadonsDeathcap', 3.8794390715667473)
('Deathblade', 3.8938555770470926)
('Zephyr', 3.901057382334022)
('BattlecastArmor', 3.901766602192147)
('DragonsClaw', 3.9071502256608497)
('WarmogsArmor', 3.907672469374576)
('InfiltratorsTalons', 3.9077498388136767)
('BladeoftheRuinedKing', 3.9144229529335997)
('IonicSpark', 3.9150741457124587)
('Morellonomicon', 3.9150741457124587)
('ZzrotPortal', 3.9156415215989777)
('RunaansHurricane', 3.91564152159898)
('GuinsoosRageblade', 3.917182462927156)
('BrambleVest', 3.9176208897485636)
('Bloodthirster', 3.917672469374609)
('ZekesHerald', 3.917672469374609)
('RebelMedal', 3.918762088974839)
('GiantSlayer', 3.919142488716969)
('Hexte